In [ ]:
!pip uninstall numpy motmetrics

Found existing installation: numpy 2.3.3
Uninstalling numpy-2.3.3:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/numpy-config
    /usr/local/lib/python3.12/dist-packages/numpy-2.3.3.dist-info/*
    /usr/local/lib/python3.12/dist-packages/numpy.libs/libgfortran-040039e1-0352e75f.so.5.0.0
    /usr/local/lib/python3.12/dist-packages/numpy.libs/libquadmath-96973f99-934c22de.so.0.0.0
    /usr/local/lib/python3.12/dist-packages/numpy.libs/libscipy_openblas64_-8fb3d286.so
    /usr/local/lib/python3.12/dist-packages/numpy/*
Proceed (Y/n)? Y
  Successfully uninstalled numpy-2.3.3
Found existing installation: motmetrics 1.4.0
Uninstalling motmetrics-1.4.0:
  Would remove:
    /usr/local/lib/python3.12/dist-packages/motmetrics-1.4.0.dist-info/*
    /usr/local/lib/python3.12/dist-packages/motmetrics/*
Proceed (Y/n)? Y
  Successfully uninstalled motmetrics-1.4.0


In [ ]:
!pip cache purge

Files removed: 12


In [ ]:
!pip install "numpy==2.0.2"
!pip install git+https://github.com/cheind/py-motmetrics.git

  Cloning https://github.com/cheind/py-motmetrics.git to /tmp/pip-req-build-lertffjq
  Running command git clone --filter=blob:none --quiet https://github.com/cheind/py-motmetrics.git /tmp/pip-req-build-lertffjq
  Resolved https://github.com/cheind/py-motmetrics.git to commit 057504b5bc6c8e9b1ef336e76cb6adcfbd30f7b0
  Preparing metadata (setup.py) ... done
  Created wheel for motmetrics: filename=motmetrics-1.4.0-py3-none-any.whl size=165446 sha256=831e3291958f9970b5068eefe38dea9cc87e9aec964a1a83703da4293655efe1
  Stored in directory: /tmp/pip-ephem-wheel-cache-sbjaan8o/wheels/14/94/6d/502b7fd4aad7a8626b6d46dc95f53655b23a80931e7a34d650
Successfully built motmetrics


In [ ]:
import motmetrics as mm
import pandas as pd
import numpy as np

In [ ]:
def calculate_metrics(gt_file, pred_file):
    """
    Calculates MOTA and IDF1 scores given ground truth and prediction files.
    """
    # 1. Load the data using pandas
    gt_df = pd.read_csv(gt_file, header=None)
    pred_df = pd.read_csv(pred_file)

    # 2. Create an accumulator
    acc = mm.MOTAccumulator(auto_id=True)

    # --- FIX: Iterate over all unique frames from both GT and Predictions ---
    all_frames = sorted(list(set(gt_df[0].unique()) | set(pred_df['frame'].unique())))

    for frame_id in all_frames:
        # Get ground truth for the current frame (if it exists)
        gt_frame = gt_df[gt_df[0] == frame_id]
        if not gt_frame.empty:
            gt_ids = gt_frame[1].values
            gt_boxes = gt_frame[[2, 3, 4, 5]].values  # Format: [x, y, w, h]
        else:
            gt_ids = np.array([])
            gt_boxes = np.empty((0, 4))

        # Get predictions for the current frame (if it exists)
        pred_frame = pred_df[pred_df['frame'] == frame_id]
        if not pred_frame.empty:
            pred_ids = pred_frame['id'].values
            # Convert predictions from [x1, y1, x2, y2] to [x, y, w, h]
            pred_boxes = np.stack([
                pred_frame['x1'],
                pred_frame['y1'],
                pred_frame['x2'] - pred_frame['x1'],
                pred_frame['y2'] - pred_frame['y1']
            ], axis=1)
        else:
            pred_ids = np.array([])
            pred_boxes = np.empty((0, 4))

        # print(gt_boxes)
        # print(pred_boxes)
        # print()
        # 4. Calculate the distance matrix (IoU)
        distance_matrix = mm.distances.iou_matrix(gt_boxes, pred_boxes, max_iou=0.9)

        # 5. Update the accumulator for the current frame
        acc.update(gt_ids, pred_ids, distance_matrix)

    # 6. Compute and display the final metrics
    mh = mm.metrics.create()
    summary = mh.compute(acc, metrics=['mota', 'idf1'], name='acc_summary')
    print(mm.io.render_summary(
        summary,
        formatters=mh.formatters,
        namemap=mm.io.motchallenge_metric_names
    ))

In [ ]:
gt_file_path = 'phua_vid_1.txt'
pred_file_path = 'phua_vid_1 annotated.csv'

calculate_metrics(gt_file_path, pred_file_path)

             MOTA  IDF1
acc_summary 51.0% 60.8%


In [ ]:
import pandas as pd

# --- Configuration ---
# Replace with the name of your CSV file
input_filename = '/content/phua_vid_3 annotated.csv'
# The name of the new file that will be created
output_filename = '/content/phua_vid_3 reordered.csv'

# --- Script ---
try:
    # Read the original CSV file into a pandas DataFrame
    df = pd.read_csv(input_filename)
    print(f"Successfully loaded '{input_filename}'.")

    # Create a list to hold the reordered data for each frame
    reordered_frames = []

    # Group the DataFrame by the 'frame' column and iterate through each frame
    for frame_num, frame_group in df.groupby('frame'):

        # Find the specific rows for ID 1 and ID 2
        row_id1 = frame_group[frame_group['id'] == 1]
        row_id2 = frame_group[frame_group['id'] == 2]

        # Select all other rows that are not ID 1 or 2
        other_rows = frame_group[~frame_group['id'].isin([1, 2])]

        # Check if both ID 1 and ID 2 exist in this frame to perform the swap
        if not row_id1.empty and not row_id2.empty:
            # Concatenate in the desired order: ID 1, then ID 2, then others
            new_frame_order = pd.concat([row_id1, row_id2, other_rows])
            reordered_frames.append(new_frame_order)
        else:
            # If one or both are missing, just keep the original order for that frame
            reordered_frames.append(frame_group)

    # Combine all the reordered frames back into a single DataFrame
    final_df = pd.concat(reordered_frames)

    # Save the final DataFrame to a new CSV file, without the extra index column
    final_df.to_csv(output_filename, index=False)

    print(f"✅ Success! The reordered data has been saved to '{output_filename}'.")

except FileNotFoundError:
    print(f"❌ Error: The file '{input_filename}' was not found. Please make sure it's in the same directory as the script.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Successfully loaded '/content/phua_vid_3 annotated.csv'.
✅ Success! The reordered data has been saved to '/content/phua_vid_3 reordered.csv'.
